# Experiments

This notebook presents the experiments described in the paper:

**The Forward-Backward Embedding of Directed Graphs**

It compares various embeddings for a clustering task on both the directed graphs and the bipartite graphs of the [Konect](http://konect.uni-koblenz.de/) collection. The datasets are downloaded automatically from this Web site.

The notebook was tested with Anaconda3. 

Make sure you have numpy, pandas, scipy and scikit-learn packages installed before running it.

In [ ]:
import urllib.request
import os
import tarfile
import glob
import shutil
import signal

In [ ]:
import numpy as np
import pandas as pd

from scipy import sparse
from sklearn.cluster.bicluster import SpectralCoclustering
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import normalize
from time import time

from metrics import cocitation_modularity
from forwardbackward_embedding import ForwardBackwardEmbedding
from spectral_embedding import SpectralEmbedding

## Processing datasets

In [ ]:
def import_graph_df(dataset, url="http://konect.uni-koblenz.de/downloads/tsv/", compression="bz2"):
    """
    Fetches a tsv file from the konect website and returns the edgelist in a pandas dataframe.
    Parameters
    ----------
    dataset: str
        the name of the file to download, without extensions
    """
    dataset_filename = dataset + ".tar." + compression
    download = urllib.request.urlretrieve(url + dataset_filename, dataset_filename)
    tf = tarfile.open(dataset_filename, "r:" + compression) 
    tf.extractall()
    os.chdir(dataset)
    for filename in glob.glob('out.*'):
        f = open(filename)
        line = f.readline()
        graph_type = line.split(' ')[2][:-1]
        graph_df = pd.read_table(f,sep = '\s+',names = ['source','target','weight','time'],comment='%')    
        f.close()
    tf.close()
    os.chdir('..')  
    os.remove(dataset_filename)
    shutil.rmtree(dataset)
    return graph_df, graph_type

In [ ]:
class Dataset:
    def __init__(self, name, directed=True, bipartite=False):        
        # get the graph
        self.name_ = name
        self.directed = directed
        self.bipartite = bipartite
        
        self.df, self.type = import_graph_df(name)
        row, col, data = self.df['source'].values, self.df['target'].values, np.ones(len(self.df))      
        self.n_edges = len(data)
             
        if bipartite:
            self.raw_adj = sparse.csr_matrix((data, (row, col)))[1:,1:]
            self.sym_adj = sparse.bmat([[None, self.raw_adj], [self.raw_adj.T, None]], format='csr')
            self.n_nodes = self.raw_adj.shape
        else:
            self.n_nodes = max(max(row), max(col))
            self.raw_adj = sparse.csr_matrix((data, (row, col)), shape=(self.n_nodes+1, self.n_nodes+1))[1:,1:]
            self.sym_adj = self.raw_adj.maximum(self.raw_adj.T)            
        
        self.n_clusters = None
 
    def display(self):
        print(self.name_+": {} nodes, {:d} edges, directed: {}.".format(self.n_nodes, self.n_edges, self.directed))
        
    def cocitation_modularity(self, partition):
        return cocitation_modularity(partition, self.raw_adj)

## Collecting datasets

### Bipartite graphs

In [ ]:
bipartite_collection = pd.read_csv('konect_bipartite.csv', sep=';')
bipartite_collection

In [ ]:
def get_bipartite_datasets(selected_codes = 'all'):
    bipartite_datasets = []
    if selected_codes == 'all':
        list_name = bipartite_collection['Filename'].values
    else:
        list_name = bipartite_collection['Filename'][bipartite_collection['Code'].isin(selected_codes)]
    for i, filename in enumerate(list_name):
        print(filename)
        bipartite_datasets.append(Dataset(filename, False, True))
    return sorted(bipartite_datasets, key=lambda x: x.n_edges)  

### Directed graphs

In [ ]:
directed_collection = pd.read_csv('konect_directed.csv', sep=';')
directed_collection

In [ ]:
def get_directed_datasets(selected_codes = 'all'):
    directed_datasets = []
    if selected_codes == 'all':
        list_name = directed_collection['Filename'].values
    else:
        list_name = directed_collection['Filename'][directed_collection['Code'].isin(selected_codes)]
    for i, filename in enumerate(list_name):
        print(filename)
        directed_datasets.append(Dataset(filename, True, False))
    return sorted(directed_datasets, key=lambda x: x.n_edges)  

## Experimental setting

In [ ]:
class Timeout():
    """Timeout class using ALARM signal."""
    class Timeout(Exception):
        pass
 
    def __init__(self, sec):
        self.sec = sec
 
    def __enter__(self):
        signal.signal(signal.SIGALRM, self.raise_timeout)
        signal.alarm(self.sec)
 
    def __exit__(self, *args):
        signal.alarm(0)    # disable alarm
 
    def raise_timeout(self, *args):
        raise Timeout.Timeout()

In [ ]:
def benchmark(datasets, algo, max_time=1000, n_clusters = 10, n_runs=1, output=None):
    """
    Evaluates an algorithm against several datasets by computing the cocitation modularity of the resulting clustering.
    
    Parameters
    ----------
    datasets: list of dataset objects
    algo: function
        algorithm to evaluate, must take a dataset as input and return an array of length dataset.n_nodes
    max_time: int or float
        maximum time in seconds allowed to algo to return an output
    n_clusters: int
        number of clusters to compute
    n_runs: int
        number of runs to perform for the same tuple(dataset, algo), results are then averaged
    output: str
        name of the output file to save results
    """
    
    if output:
        output_file = open(output, 'w')
        output_file.write('max_time = {}, n_clusters = {}, n_runs = {}\n'.format(max_time, n_clusters, n_runs))
    for dataset in datasets:
        if output:
            output_file.write(dataset.name_ + '\n')
        dataset.display()
        dataset.n_clusters = n_clusters
        avg_time, avg_mod = np.zeros(n_runs), np.zeros(n_runs)
        for i in range(n_runs):
            start_time = time()
            has_finished = False
            try:
                with Timeout(max_time):
                    y_pred = algo(dataset)
                    has_finished = True
            except Timeout.Timeout:
                result = 'Timeout'
                break
            except MemoryError:
                result = 'Memory Error'
                break
            except ValueError:
                result = 'Value Error'
                break
            except:
                result = 'Convergence Error'
                break
                
            avg_time[i] = time() - start_time
            avg_mod[i] = dataset.cocitation_modularity(y_pred)

        if has_finished:
            result = 'Average runnig time {:.2f}s. Cocitation modularity: avg = {:.2f}'\
            .format(np.mean(avg_time), np.mean(avg_mod))
        if output:
            output_file.write(result+'\n')
            output_file.write('\n')
        print(result)
        print()

## Embeddings

In [ ]:
def cluster_Id(dataset):
    """
    K-Means on the raw data, i.e, no embedding.
    """
    noembed = MiniBatchKMeans(n_clusters=dataset.n_clusters, batch_size=20000, n_init=10)
    return noembed.fit_predict(dataset.raw_adj)

In [ ]:
def cluster_Dh(dataset):
    """
    Dhillon's spectral co-clustering.
    """
    cocluster = SpectralCoclustering(n_clusters=dataset.n_clusters, svd_method='randomized')
    cocluster.fit(dataset.raw_adj)
    return cocluster.row_labels_

In [ ]:
def cluster_LE(dataset):
    """
    Spectral clustering with K-Means and Laplacian Eigenmaps
    """
    kmeans = MiniBatchKMeans(n_clusters=dataset.n_clusters, batch_size=20000, n_init=10)
    lapeigenmaps = SpectralEmbedding(dataset.n_clusters)
    lapeigenmaps.fit(dataset.sym_adj)
    if dataset.bipartite:
        return kmeans.fit_predict(lapeigenmaps.embedding_)[:dataset.n_nodes[0]]
    else:
        return kmeans.fit_predict(lapeigenmaps.embedding_)

In [ ]:
def cluster_FB(dataset):
    """
    Spectral clustering with K-Means and ForwardBackward embedding
    """
    kmeans = MiniBatchKMeans(n_clusters=dataset.n_clusters, batch_size=20000, n_init=10)
    forwardbackward = ForwardBackwardEmbedding(dataset.n_clusters)
    forwardbackward.fit(dataset.raw_adj)
    return kmeans.fit_predict(normalize(forwardbackward.embedding_))

## Results

### Directed graphs

In [ ]:
# For all datasets:
# directed_datasets = get_directed_datasets()

directed_datasets = get_directed_datasets(['MS','Mg'])

In [ ]:
benchmark(directed_datasets, cluster_Id)

In [ ]:
benchmark(directed_datasets, cluster_LE)

In [ ]:
benchmark(directed_datasets, cluster_FB)

### Bipartite graphs

In [ ]:
# For all datasets:
# bipartite_datasets = get_bipartite_datasets()

bipartite_datasets = get_bipartite_datasets(['AC','YG'])

In [ ]:
benchmark(bipartite_datasets, cluster_Id)

In [ ]:
benchmark(bipartite_datasets, cluster_Dh)

In [ ]:
benchmark(bipartite_datasets, cluster_LE)

In [ ]:
benchmark(bipartite_datasets, cluster_FB)

## Additional embedding

The experiments can also be run on the node2vec embedding. 

The node2vec package can be downloaded from [here](https://github.com/eliorc/node2vec), or using the command: **pip install node2vec**

In [ ]:
# import networkx as nx
# from node2vec import Node2Vec

In [ ]:
def cluster_N2V(dataset):
    """
    Clustering with K-Means and Node2Vec embedding
    """
    kmeans = MiniBatchKMeans(n_clusters=dataset.n_clusters, batch_size=20000, n_init=10)
    graph = nx.from_scipy_sparse_matrix(dataset.sym_adj)
    node2vec = Node2Vec(graph, dimensions=dataset.n_clusters, walk_length=5, num_walks=5, workers=4) 
    model = node2vec.fit(window=10, min_count=1, batch_words=4)
    
    if dataset.bipartite:
        n_nodes = dataset.n_nodes[0]
    else:
        n_nodes = dataset.n_nodes
    n2v_embedding = np.zeros((n_nodes, dataset.n_clusters))
    for i in range(n_nodes):
        n2v_embedding[i] = model.wv[str(i)]
    return kmeans.fit_predict(n2v_embedding)

In [ ]:
#benchmark(directed_datasets, cluster_N2V)

In [ ]:
#benchmark(bipartite_datasets, cluster_N2V)